In [1]:
import os
from concurrent.futures import ProcessPoolExecutor
from scripts.constants import create_scenes_df, dem_fill
from scripts.constants import flow_direction, flow_accumulation
import multiprocessing
from osgeo import gdal
from whitebox_workflows import WbEnvironment

In [2]:
logical_cores = multiprocessing.cpu_count()
# determine number of threads to use for multiprocessing
num_workers = int(logical_cores * 0.8)  # rounds down in case not a whole number
print(f'Number of threads to use: {num_workers}')

scenes_df = create_scenes_df()

Number of threads to use: 8


In [ ]:
# Setup whitebox_workflows environment
wbe = WbEnvironment()
wbe.max_procs = num_workers
wbe.verbose = True

In [ ]:

data_folder = 'data'

working_directory = os.getcwd()

temp_dir = 'data/temp_dir'

if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)
else:
    print(f'{temp_dir} already exists')

data/temp_dir already exists


In [4]:
# create variables
input_dem = os.path.join(data_folder, scenes_df.at[0, 'scene_folder'],scenes_df.at[0, 'input_dem'])
print(f'source dem: {input_dem}')

# grab shapefile to clip the dem
cutline_shapefile = os.path.join(data_folder, scenes_df.at[0, 'scene_folder'], scenes_df.at[0, 'shapefile'])
# get the shapefile's layer name
cutline_layer = os.path.splitext(os.path.basename(cutline_shapefile))[0]

dem_clipped = os.path.join(temp_dir, 'dem_clipped.tif')
print(f'Shapefile: {cutline_shapefile}\nShape Layer: {cutline_layer}\nOutput file: {dem_clipped}')

# processing output files
dem_pit_filled = os.path.join(temp_dir, 'pit_filled_dem.tif')
dem_filled = os.path.join(temp_dir, 'dem_filled.tif')
dem_flow = f'{temp_dir}/dem_flow.tif'
dem_accum = f'{temp_dir}/dem_accum.tif'

source dem: data/S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A/S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A_dem.tif
Shapefile: data/S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A/S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A_shape.shp
Shape Layer: S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A_shape
Output file: data/temp_dir/dem_clipped.tif


If you inspect the DEM, which is the same in both folders, it is much larger in area converage than the scenes.  Because of this extra area, we would be processing data unnecessarily, so I'm going to geometrically clip DEM file to the scene grid.

We will use the Geospatial Data Abstraction Library ([GDAL](https://gdal.org/en/stable/index.html), pronounced *\`gee doll\`*).

The documentation is difficult to navigate.   Checkout this [cookbook](https://pcjericks.github.io/py-gdalogr-cookbook/).  

In [5]:
if not os.path.exists(dem_clipped):
    gdal.Warp(
        dem_clipped, input_dem,                     # output file, input file
        format="GTiff",                             # output format - Cloud-Optimized Geotiff
        cutlineDSName=cutline_shapefile,            # Geometry to use for clipping
        cutlineLayer=cutline_layer                  # using multiple cpus to perform task
    )
else:
    print(f'File {dem_clipped} already exists.')

File data/temp_dir/dem_clipped.tif already exists.


I get errors with python 13 and numpy when trying to read the dem image.  These next set let's me check the datatype and what is assigned to nodata.

In [6]:
# open raster for reading
dataset = gdal.Open(dem_clipped)
band = dataset.GetRasterBand(1)

# Get the raster data type
dtype = band.DataType
nodata_value = band.GetNoDataValue()

print(f"Raster Data Type: {gdal.GetDataTypeName(dtype)}") 
if nodata_value is None:
    print(f'Raster NoData Value is {nodata_value}.  None is a problem, it is not a datatype')
else:
    print(f'Raster NoData Value: {nodata_value}')

dataset = None # close the file


Raster Data Type: Int16
Raster NoData Value is None.  None is a problem, it is not a datatype


/Users/ianhorn/Documents/CodeKy/pathways/data-analysis/codeky-da-capstone/venv/lib/python3.13/site-packages/osgeo/gdal.py:314: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [7]:
get_dem_clipped = wbe.read_raster(dem_clipped)

dem_pit_filled = wbe.fill_pits(get_dem_clipped)
dem_pit_filled_output = f'{temp_dir}/dem_pit_filled.tif'

wbe.write_raster(dem_pit_filled, dem_pit_filled_output)

Fill pits

fill depressions

In [11]:
# fill depressions
if not os.path.exists(f'{temp_dir}/dem_filled.tif'):
    dem_pit_filled = f'{temp_dir}/dem_pit_filled.tif'
    fill_dem = wbe.read_raster(dem_pit_filled)
    dem_filled_depressions = wbe.fill_depressions(fill_dem)
    output_file = os.path.join(temp_dir, 'dem_filled.tif')
    dem_filled_depressions = wbe.write_raster(dem_filled_depressions, output_file)
else:
    print('Filled Dem already exists.')


Filled Dem already exists.


#### Generate Flow direction
Create a flow direction model from the Filled Dem we just created.
Runs in seconds

In [ ]:
dem_fill = f'{temp_dir}/dem_fill.tif'

if not os.path.exists(f'{temp_dir}/dem_flow.tif'):
    flow_direction(dem_fill, temp_dir)

else:
    print(f'flow_direction already exists.')

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<string>", line 1, in <module>
    import sys; sys.path.insert(0, r'/Users/ianhorn/Documents/CodeKy/pathways/data-analysis/codeky-da-capstone/venv/lib/python3.13/site-packages/debugpy/_vendored/pydevd'); import pydevd; pydevd.config('http_json', 'debugpy-dap'); pydevd.settrace(host='127.0.0.1', port=49518, suspend=False, trace_only_current_thr

KeyboardInterrupt: 

#### Generate Flow Accumulation

In [ ]:
dem_flow = 'f{temp_dir}/dem_flow.tif'

if not os.path.exists('data/temp_dir/dem_accum.tif'):
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        future = executor.submit(flow_accumulation, dem_flow, temp_dir)
else:
    print('Flow accumulation file already exists.')
